In [3]:
import pandas as pd
import os
import shutil
import subprocess

In [4]:
file_path = 'PN00-1_data.txt'
df = pd.read_csv(file_path)

General names
- 00_Fp1_General  BLACK
- 00_O1_General   RED
- 00_Cz_General   GREEN
- 00_Fp2_General  BLUE
- 00_O2_General   Cyan

Pre-ictal names
- 00_Fp1_Pre-ictal  BLACK
- 00_O1_Pre-ictal   RED
- 00_Cz_Pre-ictal   GREEN
- 00_Fp2_Pre-ictal  BLUE
- 00_O2_Pre-ictal   Cyan

Ictal names
- 00_Fp1_Ictal      BLACK
- 00_O1_Ictal       RED
- 00_Cz_Ictal       GREEN
- 00_Fp2_Ictal      BLUE
- 00_O2_Ictal       Cyan

Post-ictal names
- 00_Fp1_Post-ictal  BLACK
- 00_O1_Post-ictal   RED
- 00_Cz_Post-ictal   GREEN
- 00_Fp2_Post-ictal  BLUE
- 00_O2_Post-ictal   Cyan

In [5]:
channels_columns = [
    ['1', 'C1'],   # B -> channel 1,  Fp1 Black
    ['5', 'C5'],   # F -> channel 5,  O1 Red
    ['15', 'C15'], # P -> channel 15, Cz Green
    ['17', 'C17'], # R -> channel 17, Fp2 Blue
    ['21', 'C21']  # V -> channel 21  O2 Cyan
]

In [6]:
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split('.'))
    return h * 3600 + m * 60 + s

In [10]:
#Seizure n 1
#File name: PN00-1.edf
#Registration start time: 19.39.33
#Registration end time:  20.22.58
#Seizure start time: 19.58.36
#Seizure end time: 19.59.46

registration_start_time_sec = time_to_seconds('19.39.33')
seizure_start_time_sec = time_to_seconds('19.58.36')
seizure_end_time_sec = time_to_seconds('19.59.46')

start_line_ictal = (seizure_start_time_sec - registration_start_time_sec) * 512 - 2 # Taking the first two lines out of the count
end_line_ictal =   (seizure_end_time_sec - registration_start_time_sec) * 512   - 2 # Taking the first two lines out of the count

# Show the results
print(f"Start Line Ictal: {start_line_ictal}")
print(f"End Line Ictal: {end_line_ictal}")

Start Line Ictal: 391678
End Line Ictal: 422398


## Cutting the ICTAL series

In [ ]:
result_folder = 'ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_ictal:end_line_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the GENERAL series

In [5]:
start_line_general = (start_line_ictal + end_line_ictal)/2 - 76800 # Calculating the general starting point based on 0.2.30 (2 minutes and 30 seconds) before the ictal starts
end_line_general = (start_line_ictal + end_line_ictal)/2 + 76800   # Calculating the general ending point based on 0.2.30 (2 minutes and 30 seconds) after the ictal ends

result_folder = 'GENERAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_general:end_line_general, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the PRE ICTAL series

In [6]:
start_line_pre_ictal = start_line_ictal - (abs(start_line_ictal - end_line_ictal))  # Calculating the start of the pre-ictal based on the start of the ictal
end_line_pre_ictal = start_line_ictal 

result_folder = 'PRE_ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_pre_ictal:end_line_pre_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the POST ICTAL series

In [7]:
start_line_post_ictal = end_line_ictal 
end_line_post_ictal = end_line_ictal + (abs(start_line_ictal - end_line_ictal)) # Calculating the end of the post-ictal based on the end of the ictal

result_folder = 'POST_ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_post_ictal:end_line_post_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

# Running DFA

In [8]:
def run_dfa_exe(directory, input_files, output_files):
    dfa_exe_path = 'dfa12.exe'
    
    command = [dfa_exe_path]
    
    input_str = (
        f"{input_files[0]}\n"
        f"{input_files[1]}\n"
        f"{output_files[0]}\n"
        f"{output_files[1]}\n"
    )
    
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=input_str)
    
    if process.returncode != 0:
        print(f"Erro ao executar dfa12.exe na pasta {directory}: {stderr}")
    else:
        print(f"Saída do dfa12.exe na pasta {directory}:\n{stdout}")

def process_directory(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.txt')]
    files.sort() 

    for i in range(0, len(files), 2):
        if i + 1 < len(files):
            input_files = [os.path.join(directory, files[i]), os.path.join(directory, files[i + 1])]
            output_files = [os.path.join(directory, files[i].replace('.txt', '.dat')), os.path.join(directory, files[i + 1].replace('.txt', '.dat'))]

            print("input_files a:", input_files)
            print("output_files a:", output_files)
            run_dfa_exe(directory, input_files, output_files)
        else:
            input_files = [os.path.join(directory, files[i]), os.path.join(directory, files[i])]
            output_files = [os.path.join(directory, files[i].replace('.txt', '.dat'))] * 2

            print("input_files b:", input_files)
            print("output_files b:", output_files)
            run_dfa_exe(directory, input_files, output_files)

directories = ['GENERAL', 'ICTAL', 'PRE_ICTAL', 'POST_ICTAL']

for directory in directories:
    if os.path.exists(directory):
        process_directory(directory)
    else:
        print(f"Pasta {directory} não encontrada.")


input_files a: ['GENERAL\\C1.txt', 'GENERAL\\C15.txt']
output_files a: ['GENERAL\\C1.dat', 'GENERAL\\C15.dat']
